In [1]:
import syft as sy
import torch as th
from torch import nn, optim

W0722 21:58:08.912603 4321358720 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'


W0722 21:58:08.948932 4321358720 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [2]:
hook = sy.TorchHook(th)

bob           = sy.VirtualWorker(hook, id='bob')
alice         = sy.VirtualWorker(hook, id='alice')
secure_worker = sy.VirtualWorker(hook, id='secure_worker')

In [3]:
# Dataset
bob.clear_objects()
alice.clear_objects()
secure_worker.clear_objects()

data   = th.tensor([[0.,0], [0,1], [1,0], [1,1]], requires_grad=True)
target = th.tensor([[0.], [0], [1], [1]], requires_grad=True)

bobs_data   = data[:2].send(bob)
bobs_target = target[:2].send(bob)

alices_data   = data[2:].send(alice)
alices_target = target[2:].send(alice)

In [4]:

def model_learning(obj_model, obj_opt, obj_data, obj_target):
    obj_opt.zero_grad()
    obj_pred = obj_model(obj_data)
    obj_pred = obj_pred.get().float_precision()

    obj_loss = ((obj_pred - obj_target)**2).sum()

    obj_loss.backward()
    
    obj_opt.step()

    return obj_loss.data

In [5]:
# model
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 20)
        self.fc2 = nn.Linear(20, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

# A Toy Model
model = Net()


In [6]:
model = model.fix_precision().share(alice, bob, crypto_provider=secure_worker)
data  = data.fix_precision().share(alice, bob, crypto_provider=secure_worker)

for round_iter in range(10):
    print('round_iter:', round_iter)
    
    loss   = th.FloatTensor
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for i in range(10):
        loss = model_learning(model, opt, data, target)
    else:
        print('Loss:', str(loss))
print('model:', model)